In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import xlsxwriter

In [2]:
# получение списка всех регионов
url = 'https://education.yandex.ru/universities/'
text = requests.get(url).text

p = re.compile(r'\"id\":(\d{3}),\"title\":\"([\w ()-—]+)\",\"geoId\"')
pairs = p.findall(text)

regions = {}
regions_json_form = {}
for key, value in pairs:
    key = int(key)
    regions.update({key: value})
    regions_json_form.update({key: {'name': value, 'universities': []}})

regions
#regions_json_form

{438: 'Алтайский край',
 439: 'Амурская область',
 440: 'Архангельская область',
 441: 'Астраханская область',
 442: 'Белгородская область',
 443: 'Брянская область',
 444: 'Владимирская область',
 445: 'Волгоградская область',
 446: 'Вологодская область',
 447: 'Воронежская область',
 448: 'Еврейская автономная область',
 449: 'Забайкальский край',
 450: 'Ивановская область',
 451: 'Иркутская область',
 452: 'Кабардино-Балкарская Республика',
 453: 'Калининградская область',
 454: 'Калужская область',
 455: 'Камчатский край',
 456: 'Карачаево-Черкесская Республика',
 457: 'Кемеровская область',
 458: 'Кировская область',
 459: 'Костромская область',
 460: 'Краснодарский край',
 461: 'Красноярский край',
 462: 'Курганская область',
 463: 'Курская область',
 464: 'Ленинградская область',
 465: 'Липецкая область',
 466: 'Магаданская область',
 467: 'Москва и Московская область',
 468: 'Мурманская область',
 469: 'Ненецкий автономный округ',
 470: 'Нижегородская область',
 471: 'Новгородс

In [3]:
# получение списка всех специальностей
url = 'https://education.yandex.ru/universities/'
text = requests.get(url).text

p = re.compile(r'\"id\":(\d{4}),\"code\":\"\d{2}.\d{2}.\d{2}\",\"title\":\"([\w .,()-:]+)\",\"educationTypes\"')
pairs = p.findall(text)

specialties = {}
specialties_id = []
for key, value in pairs:
    key = int(key)
    specialties.update({key: value})
    specialties_id.append(key)

specialties

{1478: 'Математика',
 1479: 'Прикладная математика и информатика',
 1480: 'Механика и математическое моделирование',
 1481: 'Прикладная математика',
 1482: 'Фундаментальные математика и механика',
 1483: 'Математика и компьютерные науки',
 1484: 'Фундаментальная информатика и информационные технологии',
 1485: 'Математическое обеспечение и администрирование информационных систем',
 1486: 'Прикладные математика и физика',
 1487: 'Физика',
 1488: 'Радиофизика',
 1489: 'Астрономия',
 1490: 'Химия',
 1491: 'Химия, физика и механика материалов',
 1492: 'Фундаментальная и прикладная химия',
 1493: 'Геология',
 1494: 'География',
 1495: 'Картография и геоинформатика',
 1496: 'Гидрометеорология',
 1497: 'Прикладная гидрометеорология',
 1498: 'Экология и природопользование',
 1499: 'Метеорология специального назначения',
 1500: 'Военная картография',
 1501: 'Биология',
 1502: 'Почвоведение',
 1503: 'Биоинженерия и биоинформатика',
 1504: 'Архитектура',
 1505: 'Реконструкция и реставрация архите

In [7]:
# набор функций для парсинга страницы
def get_level(td_list):
    return td_list[0].attrs['title']

def get_specialty(td_list):
    specialty = td_list[1].text
    pos = specialty.find('«') + 1
    if pos:
        specialty = specialty[pos:specialty.find('»')]
    return specialty

def get_department(td_list):
    return td_list[1].text

def get_subjects(td_list):
    subjects = []
    for subject in td_list[2].findAll('div', attrs={'class': ['year-stats-subjects-popup__subject']}):
        if subject.attrs['class'] == ['year-stats-subjects-popup__subject']:
            subjects.append(subject.text)
    return subjects

def get_budget_plan(td_list):
    budget_plan = None
    if td_list[3].find('div') != None:
        budget_plan = td_list[3].find('div', attrs={'class': ['year-stats-cell-popup__title']}).text
    return budget_plan

def get_budget(td_list):
    budget = None
    if td_list[3].find('div') != None:
        budget = {}
        budget_plan = td_list[3].find('div', attrs={'class': ['year-stats-cell-popup__title']}).text
        for row in td_list[3].findAll('div', attrs={'class': ['year-stats-places-popup__row']}):
            budget.update({row.find('span').text: row.find('div', attrs={'class': ['year-stats-places-popup__row-value']}).text})
    return budget

def get_budget_passing_score(td_list):
    budget_passing_score = None
    text = td_list[4].find('div', attrs={'class': ['year-stats-points-popup__points']}).text
    if text != '':
        budget_passing_score = text    
    return budget_passing_score

def get_paid_plan(td_list):
    paid_plan = None
    if td_list[5].find('div') != None:
        paid_plan = td_list[5].find('div', attrs={'class': ['year-stats-cell-popup__title']}).text
    return paid_plan

def get_paid(td_list):
    paid = None
    if td_list[5].find('div') != None:
        paid = {}
        paid_plan = td_list[5].find('div', attrs={'class': ['year-stats-cell-popup__title']}).text
        for row in td_list[5].findAll('div', attrs={'class': ['year-stats-places-popup__row']}):
            paid.update({row.find('span').text: row.find('div', attrs={'class': ['year-stats-places-popup__row-value']}).text})
    return paid

def get_paid_passing_score(td_list):
    paid_passing_score = None
    text = td_list[6].find('div', attrs={'class': ['year-stats-table-row__right-aligned']}).text
    if text != '':
        paid_passing_score = text
    return paid_passing_score

def get_cost(td_list):
    return td_list[7].find('div', attrs={'class': ['year-stats-table-row__right-aligned']}).text.replace(' ', '')

In [8]:
# сбор информации
url = 'https://education.yandex.ru/universities/university/'
university_id = 2394 # id МГУ
university = requests.get(url + str(university_id))
university.ok

soup = BeautifulSoup(university.text, 'html.parser')

table = soup.find('table', attrs={'class': ['year-stats__table']})
tr_list = table.findAll('tr')

data = []

for tr in tr_list:
    td_list = tr.findAll('td')
    
    if tr.attrs['class'] == ['year-stats-table-row', 'year-stats-table-row_bold', 'year-stats__data-row']:
        flag_no_department = td_list[1].text == 'Специальности, факультет для которых уточните в приемной комиссии'
        
        if not flag_no_department:
            level = get_level(td_list)
            specialty = get_specialty(td_list)
            
            flag_general = len(td_list) > 2
            
            if flag_general:
                subjects = get_subjects(td_list)
                budget_plan = get_budget_plan(td_list)
                budget = get_budget(td_list)
                budget_passing_score = get_budget_passing_score(td_list)
                paid_plan = get_paid_plan(td_list)
                paid = get_paid(td_list)
                paid_passing_score = get_paid_passing_score(td_list)
                cost = get_cost(td_list)
    
    elif tr.attrs['class'] == ['year-stats-table-row', 'year-stats__data-row']:
        if flag_no_department:
            level = get_level(td_list)
            specialty = get_specialty(td_list)
            
            department = None
        else:
            department = get_department(td_list)
         
        if not flag_general:
            subjects = get_subjects(td_list)
            budget_plan = get_budget_plan(td_list)
            budget = get_budget(td_list)
            budget_passing_score = get_budget_passing_score(td_list)
            paid_plan = get_paid_plan(td_list)
            paid = get_paid(td_list)
            paid_passing_score = get_paid_passing_score(td_list)
            cost = get_cost(td_list)
    
        specialty_info = {'level': level,
                          'specialty': specialty,
                          'department': department,
                          'subjects': subjects,
                          'budget_plan': budget_plan,
                          'budget': budget,
                          'budget_passing_score': budget_passing_score,
                          'paid_plan': paid_plan,
                          'paid': paid,
                          'paid_passing_score': paid_passing_score,
                          'cost': cost}
        
        data.append(specialty_info)
    
    else:
        #throw exception
        break

table = pd.DataFrame(data)
table = table[['specialty', 'department', 'level', 'subjects', 'budget_plan', 'budget', 'budget_passing_score', 'paid_plan', 'paid', 'paid_passing_score', 'cost']]
#table = table.drop(columns=['budget', 'paid'])
table = table.set_index('specialty')
table
#writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
#table.to_excel(writer,'Sheet1')
#writer.save()


,department,level,subjects,budget_plan,budget,budget_passing_score,paid_plan,paid,paid_passing_score,cost
specialty,,,,,,,,,,
Прикладная математика и информатика,факультет вычислительной математики и кибернетики,Бакалавриат,"[информатика, математика, русский язык, физика]",309,"{'По конкурсу': '265', 'Целевой прием': '4', '...",415,90,"{'По конкурсу': '0', 'Целевой прием': '0', 'По...",None,310500
Фундаментальные математика и механика,механико-математический факультет,Специалитет,"[математика, русский язык, физика]",380,"{'По конкурсу': '336', 'Целевой прием': '0', '...",325,90,"{'По конкурсу': '0', 'Целевой прием': '0', 'По...",None,310500
Фундаментальная информатика и информационные технологии,факультет вычислительной математики и кибернетики,Бакалавриат,"[информатика, математика, русский язык, физика]",15,"{'По конкурсу': '13', 'Целевой прием': '2', 'П...",413,18,"{'По конкурсу': '0', 'Целевой прием': '0', 'По...",None,310500
Прикладные математика и физика,факультет фундаментальной физико-химической ин...,Бакалавриат,"[математика, русский язык, физика]",15,"{'По конкурсу': '14', 'Целевой прием': '0', 'П...",321,5,"{'По конкурсу': '0', 'Целевой прием': '0', 'По...",None,310500
Физика,физический факультет,Бакалавриат,"[математика, русский язык, физика]",380,"{'По конкурсу': '326', 'Целевой прием': '13', ...",318,60,"{'По конкурсу': '0', 'Целевой прием': '0', 'По...",None,310500
Астрономия,физический факультет,Специалитет,"[математика, русский язык, физика]",20,"{'По конкурсу': '13', 'Целевой прием': '0', 'П...",328,5,"{'По конкурсу': '0', 'Целевой прием': '0', 'По...",None,310500
"Химия, физика и механика материалов",факультет наук о материалах,Бакалавриат,"[математика, русский язык, физика, химия]",25,"{'По конкурсу': '23', 'Целевой прием': '0', 'П...",377,5,"{'По конкурсу': '0', 'Целевой прием': '0', 'По...",None,310500
Фундаментальная и прикладная химия,факультет фундаментальной физико-химической ин...,Специалитет,"[математика, русский язык, физика, химия]",25,"{'По конкурсу': '24', 'Целевой прием': '0', 'П...",365,5,"{'По конкурсу': '0', 'Целевой прием': '0', 'По...",None,310500
Фундаментальная и прикладная химия,химический факультет,Специалитет,"[математика, русский язык, физика, химия]",223,"{'По конкурсу': '172', 'Целевой прием': '0', '...",372,25,"{'По конкурсу': '0', 'Целевой прием': '0', 'По...",None,310500


In [9]:
# формирование запроса для получения списка университетов в регионе
regions = {438: {'name': 'Алтайский край', 'universities': []}, 439: {'name': 'Амурская область', 'universities': []}, 440: {'name': 'Архангельская область', 'universities': []}}

url = 'https://education.yandex.ru/universities/graphql'

payload = {
    "query": "query SearchQuery($year: Int!, $subjectsStats: [SubjectScoreInput!]!, $possibleEducationTypes: [EDUCATION_TYPE!]!, $specialtyIds: [Int!], $regionsIds: [Int!], $requireMilitaryReserve: Boolean, $requireMilitaryContract: Boolean, $requireDormitory: Boolean) {\n  search(input: {year: $year, subjectsStats: $subjectsStats, possibleEducationTypes: $possibleEducationTypes, specialtyIds: $specialtyIds, regionsIds: $regionsIds, requireMilitaryReserve: $requireMilitaryReserve, requireMilitaryContract: $requireMilitaryContract, requireDormitory: $requireDormitory}) {\n    id\n    title\n    abbreviation\n    city\n    showCity\n    parent {\n      id\n      title\n      abbreviation\n      __typename\n    }\n    admissions {\n      id\n      srcYear\n      subjectIds\n      budgetMinPointsSum\n      paidMinPointsSum\n      tuition\n      creativeCompetitions\n      faculties {\n        id\n        title\n        abbreviation\n        __typename\n      }\n      specialtyIds\n      __typename\n    }\n    __typename\n  }\n}\n",
    "variables": {"year": "2017", "subjectsStats": [],
                  "specialtyIds": [1478,1479,1480,1481,1482,1483,1484,1485,1486,1487,
                                   1488,1489,1490,1491,1492,1493,1494,1495,1496,1497,
                                   1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,
                                   1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,
                                   1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,
                                   1528,1529,1530,1531,1532,1533,1534,1535,1536,1537,
                                   1538,1539,1540,1541,1542,1543,1544,1545,1546,1547,
                                   1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,
                                   1558,1559,1560,1561,1562,1563,1564,1565,1566,1567,
                                   1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,
                                   1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,
                                   1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,
                                   1598,1599,1600,1601,1602,1603,1604,1605,1606,1607,
                                   1608,1609,1610,1611,1612,1613,1614,1615,1616,1617,
                                   1618,1619,1620,1621,1622,1623,1624,1625,1633,1634,
                                   1635,1626,1627,1628,1629,1630,1631,1632,1636,1637,
                                   1638,1639,1640,1644,1645,1646,1641,1642,1643,1647,
                                   1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,
                                   1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,
                                   1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,
                                   1678,1679,1680,1681,1682,1683,1684,1685,1686,1687,
                                   1688,1689,1690,1691,1692,1693,1694,1698,1699,1700,
                                   1701,1702,1703,1695,1696,1697,1704,1705,1706,1707,
                                   1708,1709,1710,1711,1712,1713,1714,1716,1717,1718,
                                   1715,1719,1720,1721,1722,1723,1724,1725,1726,1727,
                                   1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,
                                   1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,
                                   1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,
                                   1758,1759,1760,1761,1762,1763,1764,1765,1767,1769,
                                   1770,1771,1766,1768,1772],
                  "possibleEducationTypes": ["APPLIED_BACHELOR", "ACADEMIC_BACHELOR", "SPECIALTY"],
                  "regionsIds": [],
                  "requireMilitaryReserve": False, "requireMilitaryContract": False, "requireDormitory": False},
    "operationName": "SearchQuery"}

headers = {
    'Content-Type': 'application/json',
}

for region_id in regions.keys():
    payload['variables']['regionsIds'] = [region_id]
    response = requests.post(url, headers=headers,
                             data=json.dumps(payload))
    regions[region_id]['universities'].append(list(map(lambda x: x['id'], response.json()['data']['search'])))

print(regions)

{438: {'name': 'Алтайский край', 'universities': [[2211, 2213, 2209, 2212, 2640, 2208, 2210]]}, 439: {'name': 'Амурская область', 'universities': [[2216, 2236, 2291, 2215]]}, 440: {'name': 'Архангельская область', 'universities': [[2539, 2690, 2540]]}}
